In [30]:
import os
import pandas as pd
%matplotlib inline
import nltk
from datetime import datetime

In [31]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

# News Headlines Sentiment

Use the news api to pull the latest news articles for bitcoin and ethereum and create a DataFrame of sentiment scores for each coin. 

Use descriptive statistics to answer the following questions:
1. Which coin had the highest mean positive score?
2. Which coin had the highest negative score?
3. Which coin had the highest positive score?

In [32]:
# Read your api key environment variable
news_api_key = os.getenv("NEWS_API_KEY")

In [33]:
# Create a newsapi client
from newsapi import NewsApiClient
newsapi = NewsApiClient(api_key=news_api_key)

def news_dataframe(topic):
    news = newsapi.get_everything(q='{topic}',
                sources='bbc-news,the-verge',
                domains='bbc.co.uk,techcrunch.com',
                from_param='2019-09-15',
                to='2019-10-14',
                language='en',
                sort_by='relevancy',
                page=2)
    return news

news_dataframe('bitcoin')

{'status': 'ok',
 'totalResults': 67,
 'articles': [{'source': {'id': 'bbc-news', 'name': 'BBC News'},
   'author': 'https://www.facebook.com/bbcnews',
   'title': 'Aerospace industry seeks Brexit reassurance',
   'description': 'Aerospace firms say it is "vital" the UK and EU have regulatory alignment.',
   'url': 'https://www.bbc.co.uk/news/business-50008268',
   'urlToImage': 'https://ichef.bbci.co.uk/news/1024/branded_news/F35A/production/_109189226_plane.jpg',
   'publishedAt': '2019-10-10T23:01:45Z',
   'content': 'Image copyrightGetty Images\r\nThere is growing concern among key aerospace manufacturers about regulatory alignment and the ability to bring products to market after Brexit.\r\nThe firms have sought reassurance that the UK would continue to be a member of the Eu… [+3558 chars]'},
  {'source': {'id': 'bbc-news', 'name': 'BBC News'},
   'author': 'https://www.facebook.com/bbcnews',
   'title': "Rail boss urges patience after Transport for Wales' first year",
   'descrip

In [34]:
# Fetch the Bitcoin news articles

def news_dataframe(topic):
    news = newsapi.get_everything(q='{topic}',
                sources='bbc-news,the-verge',
                domains='bbc.co.uk,techcrunch.com',
                from_param='2019-09-15',
                to='2019-10-14',
                language='en',
                sort_by='relevancy',
                page=2)

    
    articles = []
    for article in news['articles']:
        try:
            title = article["title"]
            description = article["description"]
            text = article["content"]
            date = article["publishedAt"][:10]

            articles.append({
                "title": title,
                "description": description,
                "text": text,
                "date": date,
            })
        except AttributeError:
            pass

    return pd.DataFrame(articles)

news_dataframe('bitcoin')

,date,description,text,title
0,2019-10-10,"Aerospace firms say it is ""vital"" the UK and E...",Image copyrightGetty Images\r\nThere is growin...,Aerospace industry seeks Brexit reassurance
1,2019-10-14,Transport for Wales' chief executive says he u...,Image copyrightRichard Jones/MLA Photography/T...,Rail boss urges patience after Transport for W...
2,2019-09-18,Enterprise search tools have always suffered f...,Enterprise search tools have always suffered f...,Salesforce brings AI power to its search tool
3,2019-10-14,The 1998 Good Friday Agreement allows people t...,Media caption'This decision goes against Good ...,Emma De Souza: Home Office appeal of case is u...
4,2019-09-22,Painted doors brighten up Hanley in Stoke-on-T...,Image caption\r\n Qamar Soheil is a street amb...,We Are Stoke-on-Trent: How painted doors are b...
5,2019-09-25,Boris Johnson returns from New York early as P...,Media playback is unsupported on your device\r...,Parliament: MPs and peers return after court r...
6,2019-10-02,Rose Glass is honoured for debut feature Saint...,Image copyrightGetty ImagesImage caption\r\n D...,Horror film wins first-time director Rose Glas...
7,2019-10-08,"Microsoft CEO Satya Nadella, delivering the ke...","Microsoft CEO Satya Nadella, delivering the ke...",Nadella warns government conference not to bet...
8,2019-10-03,Apple’s rumored iPhone SE 2 is back in the new...,And it might arrive in early 2020\r\nPhoto by ...,The next iPhone SE will reportedly look like a...
9,2019-10-12,What are the key questions facing Scotland's d...,Image copyrightGetty Images\r\nSNP members are...,SNP conference: three questions facing the party


In [35]:
# Fetch the Ethereum news articles
all_ethereum_articles = newsapi.get_everything(q='ethereum',
                                      sources='bbc-news,the-verge',
                                      domains='bbc.co.uk,techcrunch.com',
                                      from_param='2019-09-15',
                                      to='2019-10-14',
                                      language='en',
                                      sort_by='relevancy',
                                      page=2)

In [36]:
# Create the Bitcoin sentiment scores DataFrame
def analyze_sentiment(df):
    sentiments = []
    sid = SentimentIntensityAnalyzer()
    for i in range(df.shape[0]):
        if i % 100 == 0:
            print(i)
        line = df['lines'].iloc[i]
        sentiment = sid.polarity_scores(line)
        sentiments.append([sentiment['neg'], sentiment['pos'],
                           sentiment['neu'], sentiment['compound']])
    df[['neg', 'pos', 'neu', 'compound']] = pd.DataFrame(sentiments)
    df['Negative'] = df['compound'] < -0.1
    df['Positive'] = df['compound'] > 0.1
    return df 

df_bitcoin = pd.DataFrame(all_bitcoin_articles)
#analyze_sentiment(df_bitcoin)

NameError: name 'all_bitcoin_articles' is not defined

In [0]:
# Create the ethereum sentiment scores DataFrame

In [0]:
# Describe the Bitcoin Sentiment

In [0]:
# Describe the Ethereum Sentiment

### Questions:

Q: Which coin had the highest mean positive score?

A: 

Q: Which coin had the highest compound score?

A: 

Q. Which coin had the highest positive score?

A: 

---

# Tokenizer

In this section, you will use NLTK and Python to tokenize the text for each coin. Be sure to:
1. Lowercase each word
2. Remove Punctuation
3. Remove Stopwords

In [0]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation
import re

In [0]:
# Expand the default stop words list if necessary

In [0]:
# Complete the tokenizer function
def tokenizer(text):
    """Tokenizes text."""
    
    # Create a list of the words

    # Convert the words to lowercase
    
    # Remove the punctuation
    
    # Remove the stop words
    
    # Lemmatize Words into root words
    
    return tokens


In [0]:
# Create a new tokens column for bitcoin

In [0]:
# Create a new tokens column for ethereum

---

# NGrams and Frequency Analysis

In this section you will look at the ngrams and word frequency for each coin. 

1. Use NLTK to produce the n-grams for N = 2. 
2. List the top 10 words for each coin. 

In [0]:
from collections import Counter
from nltk import ngrams

In [0]:
# Generate the Bitcoin N-grams where N=2

In [0]:
# Generate the Ethereum N-grams where N=2

In [0]:
# Use the token_count function to generate the top 10 words from each coin
def token_count(tokens, N=10):
    """Returns the top N tokens from the frequency count"""
    return Counter(tokens).most_common(N)

In [0]:
# Get the top 10 words for Bitcoin

In [0]:
# Get the top 10 words for Ethereum

# Word Clouds

In this section, you will generate word clouds for each coin to summarize the news for each coin

In [0]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [20.0, 10.0]

In [0]:
# Generate the Bitcoin word cloud

In [0]:
# Generate the Ethereum word cloud

# Named Entity Recognition

In this section, you will build a named entity recognition model for both coins and visualize the tags using SpaCy.

In [0]:
import spacy
from spacy import displacy

In [0]:
# Optional - download a language model for SpaCy
# !python -m spacy download en_core_web_sm

In [0]:
# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

## Bitcoin NER

In [0]:
# Concatenate all of the bitcoin text together

In [0]:
# Run the NER processor on all of the text

# Add a title to the document

In [0]:
# Render the visualization

In [0]:
# List all Entities

---

## Ethereum NER

In [0]:
# Concatenate all of the bitcoin text together

In [0]:
# Run the NER processor on all of the text

# Add a title to the document

In [0]:
# Render the visualization

In [0]:
# List all Entities